# Main Statefarm Redux
Here we follow the main statefarm notebook, on our reduxed dataset.

In [9]:
from theano.sandbox import cuda
cuda.use('gpu0')

In [10]:
%matplotlib inline
from __future__ import print_function, division
path = "data/state/"
#path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

ImportError: No module named utils

In [11]:
!pwd
%cd /home/ubuntu/courses/deeplearning1/nbs/kaggle_data/state-farm-redux

/home/ubuntu/courses/deeplearning1/nbs/kaggle_data/state-farm-redux
/home/ubuntu/courses/deeplearning1/nbs/kaggle_data/state-farm-redux


In [12]:
path_to_data = "/home/ubuntu/courses/deeplearning1/nbs/kaggle_data/state-farm-redux/"

In [13]:
batch_size=32

In [14]:
batches = get_batches(path_to_data +'train', batch_size=batch_size)
val_batches = get_batches(path_to_data +'valid', batch_size=batch_size*2, shuffle=False)

Found 16564 images belonging to 10 classes.
Found 5786 images belonging to 10 classes.


In [15]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path_to_data)

Found 16564 images belonging to 10 classes.
Found 5786 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [8]:
# instead of batching we can also load up all data into an array
trn = get_data(path_to_data +'train')
save_array(path_to_data +'results/trn.dat', trn)
del trn

val = get_data(path_to_data +'valid')
save_array(path_to_data +'results/val.dat', val)
del val

Found 16564 images belonging to 10 classes.


KeyboardInterrupt: 

In [10]:
val = load_array(path_to_data +'results/val.dat')
trn = load_array(path_to_data +'results/trn.dat')

MemoryError: 

## Re-run experiments
on the entire set now.

In [8]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [9]:
model = conv1(batches)

Epoch 1/2
 7584/16564 [============>.................] - ETA: 195s - loss: 0.3976 - acc: 0.8982

KeyboardInterrupt: 

### Data Augmentation

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path_to_data +'train', gen_t, batch_size=batch_size)

Found 16564 images belonging to 10 classes.


In [ ]:
model = conv1(batches)

Epoch 1/2
16564/16564 [==============================] - 413s - loss: 1.2668 - acc: 0.5934 - val_loss: 1.5601 - val_acc: 0.5149
Epoch 2/2
 8416/16564 [==============>...............] - ETA: 164s - loss: 0.7115 - acc: 0.7779

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=15, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

### Three conv + maxpool and dropout
The results from above might be good but unstable - plus we want to try more complex models

In [ ]:
from keras.preprocessing import image

In [10]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path_to_data+'train', gen_t, batch_size=batch_size)

Found 16564 images belonging to 10 classes.


In [11]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [12]:
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
16564/16564 [==============================] - 533s - loss: 2.5443 - acc: 0.2706 - val_loss: 1.7020 - val_acc: 0.4941
Epoch 2/2
16564/16564 [==============================] - 508s - loss: 1.7891 - acc: 0.4348 - val_loss: 1.5494 - val_acc: 0.4943


In [14]:
model.optimizer.lr=0.001

In [15]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/10
16564/16564 [==============================] - 509s - loss: 1.4124 - acc: 0.5376 - val_loss: 1.4612 - val_acc: 0.5316
Epoch 2/10
16564/16564 [==============================] - 508s - loss: 1.1118 - acc: 0.6287 - val_loss: 1.5438 - val_acc: 0.5648
Epoch 3/10
16564/16564 [==============================] - 508s - loss: 0.9552 - acc: 0.6809 - val_loss: 1.6172 - val_acc: 0.5461
Epoch 4/10
 2048/16564 [==>...........................] - ETA: 377s - loss: 0.7841 - acc: 0.7163

KeyboardInterrupt: 

In [16]:
model.optimizer.lr=0.00001

In [17]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/10
  288/16564 [..............................] - ETA: 447s - loss: 0.9030 - acc: 0.7188

KeyboardInterrupt: 

### Using vgg16
It's likely that we want to keep all the weights up to the end of convolutional layers, since the same filters might be useful.

In [18]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i, l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx + 1]

In [19]:
conv_model = Sequential(conv_layers)

In [20]:
# when precomputing features we need to set shuffle to false BECAUSE WE NEED FEATURES IN THE SAME ORDER FOR TRAIN, VAL AND TEST
# BECAUSE THEY WILL BE PASSED TO ANOTHER MODEL!
batches = get_batches(path_to_data + "train/", batch_size=batch_size, shuffle=False)
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(path_to_data)

Found 16564 images belonging to 10 classes.
Found 16564 images belonging to 10 classes.
Found 5786 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [21]:
test_batches = get_batches(path_to_data + "test/", batch_size=batch_size, shuffle=False)
val_batches = get_batches(path_to_data + "valid/", batch_size=batch_size, shuffle=False)

Found 79726 images belonging to 1 classes.
Found 5786 images belonging to 10 classes.


In [22]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)

KeyboardInterrupt: 

In [23]:
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [23]:
print("God Daaaamn!")

God Daaaamn!


In [25]:
save_array(path_to_data+'results/conv_val_feat.dat', conv_val_feat)
save_array(path_to_data+'results/conv_test_feat.dat', conv_test_feat)
save_array(path_to_data+'results/conv_feat.dat', conv_feat)

In [24]:
conv_feat = load_array(path_to_data+'results/conv_feat.dat')
conv_val_feat = load_array(path_to_data+'results/conv_val_feat.dat')
conv_val_feat.shape

(5786, 512, 14, 14)

In [25]:
conv_test_feat = load_array(path_to_data + "results/conv_test_feat.dat")

#### batchnorm on pretrained conv layers
We have effectively taken the vgg conv layers as they were and used them to precompute features from our train/test/val batches. We need to take those predictions and use them as input to make predictions about our 10 classes.

In [43]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape = conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation="softmax"),
    ]

In [44]:
p = 0.8

In [45]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

In [32]:
# MEMORY USAGE
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('conv_test_feat', 32002654352),
 ('conv_feat', 6648922256),
 ('conv_val_feat', 2322546832),
 ('trn_labels', 1325232),
 ('test_filenames', 651360),
 ('val_labels', 462992),
 ('filenames', 140592),
 ('trn_classes', 66352),
 ('val_filenames', 48472),
 ('val_classes', 23240),
 ('Activation', 904),
 ('Adam', 904),
 ('AtrousConv1D', 904),
 ('AtrousConv2D', 904),
 ('AtrousConvolution1D', 904),
 ('AtrousConvolution2D', 904),
 ('AveragePooling1D', 904),
 ('AveragePooling2D', 904),
 ('AveragePooling3D', 904),
 ('BatchNormalization', 904),
 ('Bidirectional', 904),
 ('Conv1D', 904),
 ('Conv2D', 904),
 ('Conv3D', 904),
 ('Convolution1D', 904),
 ('Convolution2D', 904),
 ('Convolution3D', 904),
 ('Cropping1D', 904),
 ('Cropping2D', 904),
 ('Cropping3D', 904),
 ('Deconv2D', 904),
 ('Deconvolution2D', 904),
 ('Dense', 904),
 ('Dropout', 904),
 ('Embedding', 904),
 ('FileLink', 904),
 ('Flatten', 904),
 ('GRU', 904),
 ('GlobalAveragePooling2D', 904),
 ('InputSpec', 904),
 ('LSTM', 904),
 ('Lambda', 90

In [33]:
# HAD TO FREE UP MEMORY
del conv_test_feat

In [34]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
            validation_data=(conv_val_feat, val_labels))

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock


Train on 16564 samples, validate on 5786 samples
Epoch 1/1
16564/16564 [==============================] - 12s - loss: 1.5518 - acc: 0.5754 - val_loss: 0.6196 - val_acc: 0.8078


In [35]:
bn_model.optimizer.lr = 0.01

In [36]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
            validation_data=(conv_val_feat, val_labels))

Train on 16564 samples, validate on 5786 samples
Epoch 1/1
16564/16564 [==============================] - 12s - loss: 0.2913 - acc: 0.9133 - val_loss: 0.5981 - val_acc: 0.7933


In [38]:
bn_model.save_weights(path_to_data + "/models/conv_bn.h5")

## Next up - precomputed conv features of augmented data followed by batchnorm

In [39]:
# the plan is to get augmented training data, get conv output on them, concatenate that with our previous conv_features and
# use it to train a slightly different batchnormed+dropout dense model. We will also use a multiplier on the amount of data
# we predict on

In [26]:
# get augmented data
# AGAIN, WIimage.ImageDataGeneratorrrrrr NEED TO NOT SHUFFLE DATA
augment_d = image.ImageDataGenerator(width_shift_range=0.01, height_shift_range=0.05, 
                                    channel_shift_range=20, shear_range=0.1, rotation_range=15)
da_batches = get_batches(path_to_data + "/train/", augment_d, batch_size=batch_size, shuffle=False)

Found 16564 images belonging to 10 classes.


In [27]:
# HERE WE USE THE TRICK OF MULTIPLYING DATA
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample * 2)

In [28]:
# save the array
save_array(path_to_data + "/results/da_conv_feat2.dat", da_conv_feat)

In [29]:
# load the array
da_conv_feat = load_array(path_to_data + "/results/da_conv_feat2.dat")

MemoryError: 

In [33]:
# concatenate the augmented conv_feat with the unaugmented ones
da_conv_feat = np.concatenate([conv_feat, da_conv_feat])

In [34]:
# we also need to get labels for the TRIPLE conv feat
da_labels = np.concatenate([trn_labels] * 3)

In [66]:
# fit the batchnorm dense model on top
def get_bn_da_layers(p):
    return [
            MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
            Flatten(),
            Dropout(p),
            Dense(256, activation="relu"),
            BatchNormalization(),
            Dropout(p),
            Dense(256, activation="relu"),
            BatchNormalization(),
            Dropout(p),
            Dense(10, activation="softmax")
    ]

In [67]:
p = 0.8

In [68]:
bn_model = Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

In [69]:
# train the model, using the augmented data conv_features
bn_model.fit(da_conv_feat, da_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 49692 samples, validate on 5786 samples
Epoch 1/1
49692/49692 [==============================] - 33s - loss: 1.9847 - acc: 0.4000 - val_loss: 0.7019 - val_acc: 0.8076


In [70]:
# bigger lr
bn_model.optimizer.lr = 0.01

# fit for a couple more epochs
bn_model.fit(da_conv_feat, da_labels, batch_size=batch_size, nb_epoch=4,
            validation_data=(conv_val_feat, val_labels))

Train on 49692 samples, validate on 5786 samples
Epoch 1/4
49692/49692 [==============================] - 33s - loss: 0.8019 - acc: 0.7277 - val_loss: 0.4961 - val_acc: 0.8391
Epoch 2/4
49692/49692 [==============================] - 33s - loss: 0.5961 - acc: 0.8033 - val_loss: 0.4377 - val_acc: 0.8495
Epoch 3/4
49692/49692 [==============================] - 33s - loss: 0.4973 - acc: 0.8386 - val_loss: 0.4305 - val_acc: 0.8612
Epoch 4/4
49692/49692 [==============================] - 33s - loss: 0.4475 - acc: 0.8562 - val_loss: 0.4247 - val_acc: 0.8515


In [71]:
# save weights
bn_model.save_weights(path_to_data + "/models/da_conv_bn_1.h5")

In [75]:
# smaller lr
bn_model.optimizer.lr = 0.0001

# fit for a couple more epochs
bn_model.fit(da_conv_feat, da_labels, batch_size=batch_size, nb_epoch=1,
            validation_data=(conv_val_feat, val_labels))

Train on 49692 samples, validate on 5786 samples
Epoch 1/1
49692/49692 [==============================] - 32s - loss: 0.3708 - acc: 0.8811 - val_loss: 0.4061 - val_acc: 0.8648


In [73]:
# save weights
bn_model.save_weights(path_to_data + "/models/da_conv_bn_2.h5")

In [74]:
# MEMORY USAGE
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('da_conv_feat', 19946766480),
 ('conv_feat', 6648922256),
 ('conv_val_feat', 2322546832),
 ('da_labels', 3975472),
 ('trn_labels', 1325232),
 ('test_filenames', 651360),
 ('val_labels', 462992),
 ('filenames', 140592),
 ('trn_classes', 66352),
 ('val_filenames', 48472),
 ('val_classes', 23240),
 ('Activation', 904),
 ('Adam', 904),
 ('AtrousConv1D', 904),
 ('AtrousConv2D', 904),
 ('AtrousConvolution1D', 904),
 ('AtrousConvolution2D', 904),
 ('AveragePooling1D', 904),
 ('AveragePooling2D', 904),
 ('AveragePooling3D', 904),
 ('BatchNormalization', 904),
 ('Bidirectional', 904),
 ('Conv1D', 904),
 ('Conv2D', 904),
 ('Conv3D', 904),
 ('Convolution1D', 904),
 ('Convolution2D', 904),
 ('Convolution3D', 904),
 ('Cropping1D', 904),
 ('Cropping2D', 904),
 ('Cropping3D', 904),
 ('Deconv2D', 904),
 ('Deconvolution2D', 904),
 ('Dense', 904),
 ('Dropout', 904),
 ('Embedding', 904),
 ('FileLink', 904),
 ('Flatten', 904),
 ('GRU', 904),
 ('GlobalAveragePooling2D', 904),
 ('InputSpec', 904),
 ('LSTM

## Pseudolabeling
Pseudolabelling is about semi-supervised training - it's about using unlabeled data in training, by assuming that the labels our model predicts for them are good. In practice we'll use our validation set, which is weird.

Distilling knowledge is like a vanilla version of ensembling (using many good models and averaging their results) - it manages to do that without creating many models.

In [77]:
# we could use the test set instead of the val set (as it's truly unlabeled)
val_pseudo = bn_model.predict(conv_val_feat, batch_size=batch_size)

In [78]:
comb_labels = np.concatenate([da_labels, val_pseudo])

In [80]:
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

In [97]:
bn_model.load_weights(path_to_data + "/models/da_conv_bn_2.h5")

In [98]:
bn_model.fit(comb_feat, comb_labels, batch_size=batch_size, nb_epoch=1,
            validation_data=(conv_val_feat, val_labels))

Train on 55478 samples, validate on 5786 samples
Epoch 1/1
55478/55478 [==============================] - 37s - loss: 0.4711 - acc: 0.8559 - val_loss: 0.3574 - val_acc: 0.8802


In [83]:
bn_model.fit(comb_feat, comb_labels, batch_size=batch_size, nb_epoch=2,
            validation_data=(conv_val_feat, val_labels))

Train on 55478 samples, validate on 5786 samples
Epoch 1/4
55478/55478 [==============================] - 37s - loss: 0.4086 - acc: 0.8791 - val_loss: 0.3878 - val_acc: 0.8830
Epoch 2/4
55478/55478 [==============================] - 36s - loss: 0.3859 - acc: 0.8873 - val_loss: 0.3655 - val_acc: 0.8839
Epoch 3/4
55478/55478 [==============================] - 37s - loss: 0.3670 - acc: 0.8939 - val_loss: 0.3944 - val_acc: 0.8764
Epoch 4/4
55478/55478 [==============================] - 36s - loss: 0.3540 - acc: 0.8983 - val_loss: 0.3881 - val_acc: 0.8859


In [99]:
# smaller lr at the end
bn_model.optimizer.lr=0.00001

In [100]:
bn_model.fit(comb_feat, comb_labels, batch_size=batch_size, nb_epoch=2,
            validation_data=(conv_val_feat, val_labels))

Train on 55478 samples, validate on 5786 samples
Epoch 1/2
55478/55478 [==============================] - 37s - loss: 0.4149 - acc: 0.8782 - val_loss: 0.3646 - val_acc: 0.8901
Epoch 2/2
55478/55478 [==============================] - 37s - loss: 0.3867 - acc: 0.8870 - val_loss: 0.3409 - val_acc: 0.8887


In [101]:
# smaller lr at the end
bn_model.optimizer.lr=0.000001

In [102]:
bn_model.fit(comb_feat, comb_labels, batch_size=batch_size, nb_epoch=1,
            validation_data=(conv_val_feat, val_labels))

Train on 55478 samples, validate on 5786 samples
Epoch 1/1
55478/55478 [==============================] - 37s - loss: 0.3602 - acc: 0.8963 - val_loss: 0.3730 - val_acc: 0.8897


In [103]:
# we could use THE ENTIRE TRAINING SET and train on that one in this way

In [104]:
# save weights
bn_model.save_weights(path_to_data + "/models/da_conv_bn_3.h5")

## Submitting

In [105]:
# MEMORY USAGE
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('comb_feat', 22269313168),
 ('da_conv_feat', 19946766480),
 ('conv_feat', 6648922256),
 ('conv_val_feat', 2322546832),
 ('comb_labels', 4438352),
 ('da_labels', 3975472),
 ('trn_labels', 1325232),
 ('test_filenames', 651360),
 ('val_labels', 462992),
 ('val_pseudo', 231552),
 ('filenames', 140592),
 ('trn_classes', 66352),
 ('val_filenames', 48472),
 ('val_classes', 23240),
 ('Activation', 904),
 ('Adam', 904),
 ('AtrousConv1D', 904),
 ('AtrousConv2D', 904),
 ('AtrousConvolution1D', 904),
 ('AtrousConvolution2D', 904),
 ('AveragePooling1D', 904),
 ('AveragePooling2D', 904),
 ('AveragePooling3D', 904),
 ('BatchNormalization', 904),
 ('Bidirectional', 904),
 ('Conv1D', 904),
 ('Conv2D', 904),
 ('Conv3D', 904),
 ('Convolution1D', 904),
 ('Convolution2D', 904),
 ('Convolution3D', 904),
 ('Cropping1D', 904),
 ('Cropping2D', 904),
 ('Cropping3D', 904),
 ('Deconv2D', 904),
 ('Deconvolution2D', 904),
 ('Dense', 904),
 ('Dropout', 904),
 ('Embedding', 904),
 ('FileLink', 904),
 ('Flatten', 90

In [106]:
# NEED TO FREE UP MEMORY BEFORE operating on the test set
del comb_feat
del da_conv_feat
del conv_feat
del da_labels
del trn_labels

In [110]:
# why divide by 9?
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [108]:
# we need to use the model to predict val_preds, I think
val_preds = bn_model.predict(conv_val_feat, batch_size=batch_size*2)

In [139]:
# not sure if cross entropy is good when small or big
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.89)).eval()

array(0.44426681360920695)

In [118]:
conv_test_feat = load_array(path_to_data +'results/conv_test_feat.dat')

In [119]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [148]:
subm = do_clip(preds,0.95)

In [149]:
subm_name = path_to_data +'results/subm_3.csv'

In [150]:
# this is weird...
classes = sorted(batches.class_indices, key=batches.class_indices.get)
print(classes)

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']


In [151]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.071730,0.005556,0.008983,0.005556,0.005556,0.005556,0.022735,0.035755,0.135804,0.716470
1,img_14887.jpg,0.603387,0.009996,0.005556,0.005556,0.005556,0.006691,0.005556,0.005556,0.027201,0.345809
2,img_62885.jpg,0.036221,0.005556,0.005556,0.053953,0.881569,0.006774,0.005556,0.005556,0.007683,0.008447
3,img_45125.jpg,0.012130,0.005556,0.035124,0.005556,0.019442,0.005556,0.621067,0.025236,0.251576,0.029713
4,img_22633.jpg,0.059357,0.031130,0.005556,0.005556,0.005556,0.008725,0.005556,0.006533,0.247365,0.638516


In [152]:
submission.to_csv(subm_name, index=False)

In [153]:
FileLink(subm_name)

/home/ubuntu/courses/deeplearning1/nbs/kaggle_data/state-farm-redux/results/subm_3.csv

In [131]:
print("Full stop :)")

Full stop :)
